In [1]:
%pip install -q geopy python-dotenv tqdm

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import time
from tqdm import tqdm
import requests
from dotenv import load_dotenv
import os
import urllib.parse
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

load_dotenv()

pd.set_option('display.max_columns', None)  # Show all columns in DataFrame

import os
try:
    if run_once == False:
        pass
    else:
        print("os.chdir('..') already run once -- will not run again")
except:
    run_once = True
    os.chdir("..")

In [2]:
# We can load the current dataset
df = pd.read_csv("data/accidents/ny_accidents_cleaned2.csv")
df.head()

/var/folders/qh/zp3sqlm15lj3hm8lhm_7340h0000gp/T/ipykernel_14804/3100941634.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/accidents/ny_accidents_cleaned2.csv")


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,NUMBER OF VEHICLES INVOLVED
0,2023-11-01,1:29,BROOKLYN,11230.0,-73.969750,40.621810,"(-73.96975, 40.62181)",OCEAN PARKWAY,AVENUE K,NaN,1,0,0,0,0,0,1,0,Unspecified,Unspecified,Unspecified,NaN,NaN,4675373,Moped,Sedan,Sedan,NaN,NaN,3
1,2021-09-11,9:35,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0,0,0,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN,1
2,2021-12-14,8:13,BROOKLYN,11233.0,-73.917260,40.683320,"(-73.91726, 40.68332)",SARATOGA AVENUE,DECATUR STREET,NaN,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN,0
3,2021-12-14,17:05,NaN,NaN,40.709183,-73.956825,"(40.709183, -73.956825)",BROOKLYN QUEENS EXPRESSWAY,NaN,NaN,0,0,0,0,0,0,0,0,Passing Too Closely,Unspecified,NaN,NaN,NaN,4486555,Sedan,Tractor Truck Diesel,NaN,NaN,NaN,2
4,2021-12-14,8:17,BRONX,10475.0,40.868160,-73.831480,"(40.86816, -73.83148)",NaN,NaN,344 BAYCHESTER AVENUE,2,0,0,0,0,0,2,0,Unspecified,Unspecified,NaN,NaN,NaN,4486660,Sedan,Sedan,NaN,NaN,NaN,2


# Collisions Dataframe

## Columns

    - CRASH DATE: The date of the crash in MM/DD/YYYY format
    - CRASH TIME: The time of the crash in 24-hr format
    - BOROUGH: The borough of the accident ("QUEENS", "BROOKLYN", "MANHATTAN", "BRONX", "STATEN ISLAND")
        - NaN: we can potentially use LATITUDE/LONGITUDE/LOCATION/ON STREET NAME/CROSS STREET NAME/ZIP CODE
    - ZIP CODE: The 5-digit ZIP code (many NaNs)
    - LATITUDE/LONGITUDE: The coordinates as floats
    - LOCATION: LATITUDE and LONGITUDE combined, expressed as (LATITUDE, LONGITUDE)
        - We can probably remove this to reduce redundancy
    - ON STREET NAME: The street where the accident occurred
    - CROSS STREET NAME: The street of if it's at an intersection
    - NUMBER OF _ INJURED/KILLED: integer value
        - NaNs: We can probably convert them to 0, or observe the other fields
    - 

## Data Cleaning

Let's clean up the data, particularly the NaN values.

In [4]:
df.isna().sum()  # Count NaN values in each column

CRASH DATE                             0
CRASH TIME                             0
BOROUGH                           675290
ZIP CODE                          675563
LATITUDE                          240142
LONGITUDE                         240142
LOCATION                          240142
ON STREET NAME                    475858
CROSS STREET NAME                 838223
OFF STREET NAME                  1811672
NUMBER OF PERSONS INJURED             18
NUMBER OF PERSONS KILLED              31
NUMBER OF PEDESTRIANS INJURED          0
NUMBER OF PEDESTRIANS KILLED           0
NUMBER OF CYCLIST INJURED              0
NUMBER OF CYCLIST KILLED               0
NUMBER OF MOTORIST INJURED             0
NUMBER OF MOTORIST KILLED              0
CONTRIBUTING FACTOR VEHICLE 1       7657
CONTRIBUTING FACTOR VEHICLE 2     350378
CONTRIBUTING FACTOR VEHICLE 3    2036488
CONTRIBUTING FACTOR VEHICLE 4    2158965
CONTRIBUTING FACTOR VEHICLE 5    2185247
COLLISION_ID                           0
VEHICLE TYPE COD

We can observe a few things:

    - There are some NaN "NUMBER OF PERSONS KILLED/INJURED" rows, but no NaN values for the related columns that likely derive the former.

    - LATITUDE == LONGITUDE == LOCATION for NaNs

    - There's always a CRASH DATE and CRASH TIME -- that's great :)

    - VEHICLE TYPE CODE 1-5 are increasing in NaNs going from 1-5. We can likely use this to derive how many vehicles are involved in the accident

### Deriving Number of Vehicles Involved

In [5]:
# Let's derive the number of vehicles involved in each accident
vehicle_columns = [f'VEHICLE TYPE CODE {i}' for i in range(1, 6)]

df[vehicle_columns].head()
df["NUMBER OF VEHICLES INVOLVED"] = df[vehicle_columns].notna().sum(axis=1)
df["NUMBER OF VEHICLES INVOLVED"].value_counts()

NUMBER OF VEHICLES INVOLVED
2    1605588
1     421295
3     117665
4      25289
0      15718
5       9601
Name: count, dtype: int64

### Cleaning NaNs in Injuries and Deaths columns

In [6]:
# We can observe the number of NaN values in "NUMBER OF PERSONS INJURED" and "NUMBER OF PERSONS KILLED" columns

injury_columns = [
    "NUMBER OF PERSONS INJURED",
    "NUMBER OF PEDESTRIANS INJURED",
    "NUMBER OF CYCLIST INJURED",
    "NUMBER OF MOTORIST INJURED",
]

killed_columns = [
    "NUMBER OF PERSONS KILLED",
    "NUMBER OF PEDESTRIANS KILLED",
    "NUMBER OF CYCLIST KILLED",
    "NUMBER OF MOTORIST KILLED",
]

injury_df_na = df[df["NUMBER OF PERSONS INJURED"].isna()][injury_columns]
killed_df_na = df[df["NUMBER OF PERSONS KILLED"].isna()][killed_columns]
injury_df_na

,NUMBER OF PERSONS INJURED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF CYCLIST INJURED,NUMBER OF MOTORIST INJURED
361554,NaN,0,1,0
748712,NaN,0,0,1
798103,NaN,0,0,1
848142,NaN,0,0,0
891244,NaN,0,0,1
907898,NaN,0,0,0
957161,NaN,0,0,0
980926,NaN,0,0,0
983307,NaN,0,0,0
989988,NaN,0,0,0


In [7]:
# Let's aggregate the number of persons injured and killed into the NaN rows
injury_df_na["NUMBER OF PERSONS INJURED"] = (
    injury_df_na["NUMBER OF PEDESTRIANS INJURED"]
    + injury_df_na["NUMBER OF CYCLIST INJURED"]
    + injury_df_na["NUMBER OF MOTORIST INJURED"]
)
killed_df_na["NUMBER OF PERSONS KILLED"] = (
    killed_df_na["NUMBER OF PEDESTRIANS KILLED"]
    + killed_df_na["NUMBER OF CYCLIST KILLED"]
    + killed_df_na["NUMBER OF MOTORIST KILLED"]
)

injury_df_na

,NUMBER OF PERSONS INJURED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF CYCLIST INJURED,NUMBER OF MOTORIST INJURED
361554,1,0,1,0
748712,1,0,0,1
798103,1,0,0,1
848142,0,0,0,0
891244,1,0,0,1
907898,0,0,0,0
957161,0,0,0,0
980926,0,0,0,0
983307,0,0,0,0
989988,0,0,0,0


In [8]:
killed_df_na

,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST KILLED
694104,0,0,0,0
696519,0,0,0,0
722173,0,0,0,0
727971,0,0,0,0
742488,0,0,0,0
743412,0,0,0,0
748712,0,0,0,0
769544,0,0,0,0
776739,0,0,0,0
819829,0,0,0,0


In [9]:
# Let's update the original DataFrame with the aggregated values
df.loc[df["NUMBER OF PERSONS INJURED"].isna(), "NUMBER OF PERSONS INJURED"] = injury_df_na["NUMBER OF PERSONS INJURED"]
df.loc[df["NUMBER OF PERSONS KILLED"].isna(), "NUMBER OF PERSONS KILLED"] = killed_df_na["NUMBER OF PERSONS KILLED"]

# Now we can check if there are still NaN values in the injury and killed columns
df[injury_columns].isna().sum(), df[killed_columns].isna().sum()

(NUMBER OF PERSONS INJURED        0
 NUMBER OF PEDESTRIANS INJURED    0
 NUMBER OF CYCLIST INJURED        0
 NUMBER OF MOTORIST INJURED       0
 dtype: int64,
 NUMBER OF PERSONS KILLED        0
 NUMBER OF PEDESTRIANS KILLED    0
 NUMBER OF CYCLIST KILLED        0
 NUMBER OF MOTORIST KILLED       0
 dtype: int64)

In [6]:
df.isna().sum()

CRASH DATE                             0
CRASH TIME                             0
BOROUGH                           675290
ZIP CODE                          675563
LATITUDE                          240142
LONGITUDE                         240142
LOCATION                          240142
ON STREET NAME                    475858
CROSS STREET NAME                 838223
OFF STREET NAME                  1811672
NUMBER OF PERSONS INJURED              0
NUMBER OF PERSONS KILLED               0
NUMBER OF PEDESTRIANS INJURED          0
NUMBER OF PEDESTRIANS KILLED           0
NUMBER OF CYCLIST INJURED              0
NUMBER OF CYCLIST KILLED               0
NUMBER OF MOTORIST INJURED             0
NUMBER OF MOTORIST KILLED              0
CONTRIBUTING FACTOR VEHICLE 1       7657
CONTRIBUTING FACTOR VEHICLE 2     350378
CONTRIBUTING FACTOR VEHICLE 3    2036488
CONTRIBUTING FACTOR VEHICLE 4    2158965
CONTRIBUTING FACTOR VEHICLE 5    2185247
COLLISION_ID                           0
VEHICLE TYPE COD

In [68]:
# Standardize the street names to all-uppercase

df[["ON STREET NAME", "CROSS STREET NAME"]] = df[["ON STREET NAME", "CROSS STREET NAME"]].apply(lambda x: x.str.strip().str.upper())

In [13]:
# Let's convert the dtype from float to int64

df["NUMBER OF PERSONS INJURED"] = df["NUMBER OF PERSONS INJURED"].astype(np.int64)
df["NUMBER OF PERSONS KILLED"] = df["NUMBER OF PERSONS KILLED"].astype(np.int64)
df.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,NUMBER OF VEHICLES INVOLVED
0,2023-11-01,1:29,BROOKLYN,11230.0,-73.969750,40.621810,"(-73.96975, 40.62181)",OCEAN PARKWAY,AVENUE K,NaN,1,0,0,0,0,0,1,0,Unspecified,Unspecified,Unspecified,NaN,NaN,4675373,Moped,Sedan,Sedan,NaN,NaN,3
1,2021-09-11,9:35,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0,0,0,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN,1
2,2021-12-14,8:13,BROOKLYN,11233.0,-73.917260,40.683320,"(-73.91726, 40.68332)",SARATOGA AVENUE,DECATUR STREET,NaN,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN,0
3,2021-12-14,17:05,NaN,NaN,40.709183,-73.956825,"(40.709183, -73.956825)",BROOKLYN QUEENS EXPRESSWAY,NaN,NaN,0,0,0,0,0,0,0,0,Passing Too Closely,Unspecified,NaN,NaN,NaN,4486555,Sedan,Tractor Truck Diesel,NaN,NaN,NaN,2
4,2021-12-14,8:17,BRONX,10475.0,40.868160,-73.831480,"(40.86816, -73.83148)",NaN,NaN,344 BAYCHESTER AVENUE,2,0,0,0,0,0,2,0,Unspecified,Unspecified,NaN,NaN,NaN,4486660,Sedan,Sedan,NaN,NaN,NaN,2


In [59]:
# We can save our progress
df.to_csv('data/accidents/ny_accidents_cleaned1.csv', index=False)

### Cleaning missing Location values

The missing locations can be tricky to deal with. We have many missing coordinates, boroughs, street names, etc.

In [3]:
# Let's check for the NaN locations
location_columns = [
    "BOROUGH",
    "ZIP CODE",
    "LATITUDE",
    "LONGITUDE",
    "ON STREET NAME",
    "CROSS STREET NAME",
    "OFF STREET NAME",
]

# We can find rows where all location columns are NaN
# In this case, we can remove these rows since we cannot infer any location information
all_nan_locations = df[df[location_columns].isna().all(axis=1)][location_columns]

# Drop these rows from the DataFrame
df = df.drop(all_nan_locations.index)

In [9]:
# Let's check for NaN Latitude and Longitude values with non-NaN street names
nan_lat_long = df[df["LATITUDE"].isna() | df["LONGITUDE"].isna()]
# We can use the street names to infer the missing latitude and longitude values
street_columns = ["ON STREET NAME", "CROSS STREET NAME", "BOROUGH"]
non_nan_streets = nan_lat_long[nan_lat_long["ON STREET NAME"].notna()]
non_nan_streets = non_nan_streets[street_columns]

non_nan_streets

,ON STREET NAME,CROSS STREET NAME,BOROUGH
0,WHITESTONE EXPRESSWAY,20 AVENUE,NaN
1,QUEENSBORO BRIDGE UPPER,NaN,NaN
3,THROGS NECK BRIDGE,NaN,NaN
4,BROOKLYN BRIDGE,NaN,NaN
5,WEST 54 STREET,NaN,NaN
...,...,...,...
2194770,QUINCY STREET,THROOP AVE,BROOKLYN
2194787,WALWORHT STREET,MYRTLE AVE,NaN
2194861,Jerome avenue,east 162 street,NaN
2194937,Atlantic Avenue,South Portland road,NaN


In [34]:
deduplicated_intersections = non_nan_streets.dropna(axis=0).map(lambda x: x.strip().upper()) \
    .drop_duplicates(subset=["ON STREET NAME", "CROSS STREET NAME"])
    
deduplicated_intersections

,ON STREET NAME,CROSS STREET NAME,BOROUGH
136,GRAND AVENUE,GREENE AVENUE,BROOKLYN
155,49 STREET,5 AVENUE,MANHATTAN
160,VANDERVORT AVENUE,ANTHONY STREET,BROOKLYN
271,33 ST,31AVE,QUEENS
385,WEST 33 STREET,7 AVENUE,MANHATTAN
...,...,...,...
2192665,E 148 ST,ST ANNS AVE,BRONX
2192885,112 PL,ASTORIA BLVD,QUEENS
2192927,E 14 ST,1 AVE,MANHATTAN
2194105,SHORE FRONT PKWY,BEACH 73 ST,QUEENS


We will use the Here Geocoding API to geocode locations

In [42]:
import threading
import time

street_coordinates = {}
HERE_API_KEY = os.getenv("HERE_API_KEY")
api_url = f"https://geocode.search.hereapi.com/v1/geocode?q={{}}&apiKey={HERE_API_KEY}"

# Shared lock for thread safety
results_lock = threading.Lock()
rate_lock = threading.Lock()  # For rate limiting
last_request_time = 0  # Global variable, not thread-local

def geocode_intersection(street_name, cross_street_name, borough):
    """Geocode the intersection of two streets in a borough."""
    global last_request_time
    
    # Rate limiting with global timing
    with rate_lock:
        elapsed = time.time() - last_request_time
        if elapsed < 0.02:  # 20ms = 50 requests/second max
            time.sleep(0.02 - elapsed)
        last_request_time = time.time()
    
    address = f"{street_name} and {cross_street_name}, {borough}, New York"
    formatted_address = urllib.parse.quote_plus(address)
    request = api_url.format(formatted_address)
    try:
        response = requests.get(request)
        if response.status_code == 200:
            data = response.json()
            if data.get("items", []) and len(data["items"]) > 0:
                location = data["items"][0]['position']
                return (location["lat"], location["lng"])
        return (None, None)
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return (None, None)

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {}

    for index, row in tqdm(deduplicated_intersections.iterrows(),
                          total=deduplicated_intersections.shape[0],
                          desc="Geocoding Streets",
                          position=0, leave=True):
        street_name = row["ON STREET NAME"]
        cross_street_name = row["CROSS STREET NAME"]
        borough = row["BOROUGH"]
        
        future = executor.submit(geocode_intersection, street_name, cross_street_name, borough)
        futures[future] = (street_name, cross_street_name, borough)
    
    for future in tqdm(as_completed(futures),
                      total=len(futures),
                      desc="Processing Responses",
                      position=1, leave=True):
        coordinates = future.result()  # This is (lat, lng) or (None, None)
        street_name, cross_street_name, borough = futures[future]
        
        with results_lock:
            street_coordinates[(street_name, cross_street_name)] = coordinates

Processing Responses: 100%|██████████| 16822/16822 [09:35<00:00, 29.25it/s]


In [46]:
import pickle
# Save the street coordinates to a file
with open("data/accidents/street_coordinates.pkl", "wb") as f:
    pickle.dump(street_coordinates, f)

In [ ]:
# We can load our street coordinates

with open("data/accidents/street_coordinates.pkl", "rb") as f:
    street_coordinates = pickle.load(f)

In [69]:
# Now we can assign coordinates to rows with missing values but non-null street and cross street names

def apply_coordinates(row):
    """
    Given a row, get its street and cross street names,
    reference the street_coordinates dict we just computed,
    and assign it to row["LOCATION"] and row[["LATITUDE", "LONGITUDE"]]
    """
    street_name = row["ON STREET NAME"]
    cross_street_name = row["CROSS STREET NAME"]
    if (street_name, cross_street_name) in street_coordinates:
        # Get the coordinates for the street and cross street
        coordinates = street_coordinates[(street_name, cross_street_name)]
        longitude, latitude = coordinates
        row["LATITUDE"] = latitude
        row["LONGITUDE"] = longitude
        row["LOCATION"] = (latitude, longitude)
    return row

# Get the rows where street and cross street names are not null
non_null_streets = df[df[["ON STREET NAME", "CROSS STREET NAME"]].notna().all(axis=1) & df["LOCATION"].isna()]
location_columns = ["LATITUDE", "LONGITUDE", "LOCATION"]
df = df.apply(apply_coordinates, axis=1)

In [74]:
df.isna().sum()

CRASH DATE                             0
CRASH TIME                             0
BOROUGH                           645267
ZIP CODE                          645540
LATITUDE                          176064
LONGITUDE                         176064
LOCATION                          176037
ON STREET NAME                    445851
CROSS STREET NAME                 808217
OFF STREET NAME                  1781649
NUMBER OF PERSONS INJURED              0
NUMBER OF PERSONS KILLED               0
NUMBER OF PEDESTRIANS INJURED          0
NUMBER OF PEDESTRIANS KILLED           0
NUMBER OF CYCLIST INJURED              0
NUMBER OF CYCLIST KILLED               0
NUMBER OF MOTORIST INJURED             0
NUMBER OF MOTORIST KILLED              0
CONTRIBUTING FACTOR VEHICLE 1       7571
CONTRIBUTING FACTOR VEHICLE 2     347659
CONTRIBUTING FACTOR VEHICLE 3    2009940
CONTRIBUTING FACTOR VEHICLE 4    2129641
CONTRIBUTING FACTOR VEHICLE 5    2155395
COLLISION_ID                           0
VEHICLE TYPE COD

We've reduced the amount of NaN coordinates by 70000 :)

Geocoding with only `ON STREET NAME` is likely not descriptive enough, so we can drop them

In [86]:
df_cleaned_locations = df[df[location_columns].notna().all(axis=1)].reset_index()
df_cleaned_locations

,index,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,NUMBER OF VEHICLES INVOLVED
0,2,11/01/2023,1:29,BROOKLYN,11230.0,-73.969750,40.621810,"(-73.96975, 40.62181)",OCEAN PARKWAY,AVENUE K,NaN,1.0,0.0,0,0,0,0,1,0,Unspecified,Unspecified,Unspecified,NaN,NaN,4675373,Moped,Sedan,Sedan,NaN,NaN,3
1,9,09/11/2021,9:35,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,0.0,0,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN,1
2,10,12/14/2021,8:13,BROOKLYN,11233.0,-73.917260,40.683320,"(-73.91726, 40.68332)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,0.0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN,0
3,12,12/14/2021,17:05,NaN,NaN,40.709183,-73.956825,"(40.709183, -73.956825)",BROOKLYN QUEENS EXPRESSWAY,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Passing Too Closely,Unspecified,NaN,NaN,NaN,4486555,Sedan,Tractor Truck Diesel,NaN,NaN,NaN,2
4,13,12/14/2021,8:17,BRONX,10475.0,40.868160,-73.831480,"(40.86816, -73.83148)",NaN,NaN,344 BAYCHESTER AVENUE,2.0,0.0,0,0,0,0,2,0,Unspecified,Unspecified,NaN,NaN,NaN,4486660,Sedan,Sedan,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989064,2165128,07/29/2025,11:50,BROOKLYN,11231.0,40.678530,-74.001980,"(40.67853, -74.00198)",NaN,NaN,661 HENRY ST,0.0,0.0,0,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,4831425,Sedan,NaN,NaN,NaN,NaN,1
1989065,2165129,07/29/2025,11:25,MANHATTAN,10018.0,40.752280,-73.989690,"(40.75228, -73.98969)",W 36 ST,7 AVE,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,4831839,Sedan,Pick-up Truck,NaN,NaN,NaN,2
1989066,2165130,07/26/2025,11:00,QUEENS,11433.0,40.692970,-73.780235,"(40.69297, -73.780235)",SAYRES AVE,MERRICK BLVD,NaN,0.0,0.0,0,0,0,0,0,0,Following Too Closely,Unspecified,NaN,NaN,NaN,4831849,Sedan,Sedan,NaN,NaN,NaN,2
1989067,2165131,07/27/2025,2:00,NaN,NaN,40.845330,-73.923300,"(40.84533, -73.9233)",CROSS BRONX EXPY RAMP,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,4831872,Sedan,NaN,NaN,NaN,NaN,1


In [4]:
# Finally, we can convert the crash date from MM/DD/YY to YY/MM/DD
df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE'], format='%m/%d/%Y', errors='coerce').dt.strftime('%Y-%m-%d')
df['CRASH DATE'].head()

0    2023-11-01
1    2021-09-11
2    2021-12-14
3    2021-12-14
4    2021-12-14
Name: CRASH DATE, dtype: object

In [87]:
# We can save our progress again
df_cleaned_locations.to_csv("data/accidents/ny_accidents_cleaned2.csv", index=False)